<a href="https://colab.research.google.com/github/Zorug/Steam-Stuff/blob/master/BS_%26_CSV_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

comparação com os dados do site: https://www.steamcardexchange.net 

extração do csv: https://steam.tools/cards/